#import

In [43]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor

import pickle #для сохранения моделей

In [1]:
!git clone https://github.com/Lasttrader/DA_repo

Cloning into 'DA_repo'...
remote: Enumerating objects: 63, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 63 (delta 1), reused 1 (delta 0), pack-reused 58
Receiving objects: 100% (63/63), 31.42 MiB | 16.78 MiB/s, done.
Resolving deltas: 100% (14/14), done.


#загрузка данных

In [ ]:
!wget https://raw.githubusercontent.com/Lasttrader/DA_repo/main/bank.csv

In [3]:
path = '/content/DA_repo/bank.csv'

In [53]:
data = pd.read_csv(path, sep = ';')
data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


In [ ]:
data.info()

#Категориальные признаки

In [8]:
data.describe(include = 'object')

,job,marital,education,default,housing,loan,contact,month,poutcome,y
count,4521,4521,4521,4521,4521,4521,4521,4521,4521,4521
unique,12,3,4,2,2,2,3,12,4,2
top,management,married,secondary,no,yes,no,cellular,may,unknown,no
freq,969,2797,2306,4445,2559,3830,2896,1398,3705,4000


In [12]:
# список категориальных признаков
categorical_cols = data.describe(include = 'object').columns.to_list()

In [15]:
categorical_data = data[categorical_cols]
categorical_data.head()

,job,marital,education,default,housing,loan,contact,month,poutcome,y
0,unemployed,married,primary,no,no,no,cellular,oct,unknown,no
1,services,married,secondary,no,yes,yes,cellular,may,failure,no
2,management,single,tertiary,no,yes,no,cellular,apr,failure,no
3,management,married,tertiary,no,yes,yes,unknown,jun,unknown,no
4,blue-collar,married,secondary,no,yes,no,unknown,may,unknown,no


In [16]:
#label encoder
le = LabelEncoder()

for col in categorical_cols:
  categorical_data[col + 'LE'] = le.fit_transform(categorical_data[col])
  with open(f'{col}_le.pkl', 'wb') as f:
    pickle.dump(le, f)

categorical_data.head()

<ipython-input-16-828e0cebc184>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  categorical_data[col + 'LE'] = le.fit_transform(categorical_data[col])
<ipython-input-16-828e0cebc184>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  categorical_data[col + 'LE'] = le.fit_transform(categorical_data[col])
<ipython-input-16-828e0cebc184>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

,job,marital,education,default,housing,loan,contact,month,poutcome,y,jobLE,maritalLE,educationLE,defaultLE,housingLE,loanLE,contactLE,monthLE,poutcomeLE,yLE
0,unemployed,married,primary,no,no,no,cellular,oct,unknown,no,10,1,0,0,0,0,0,10,3,0
1,services,married,secondary,no,yes,yes,cellular,may,failure,no,7,1,1,0,1,1,0,8,0,0
2,management,single,tertiary,no,yes,no,cellular,apr,failure,no,4,2,2,0,1,0,0,0,0,0
3,management,married,tertiary,no,yes,yes,unknown,jun,unknown,no,4,1,2,0,1,1,2,6,3,0
4,blue-collar,married,secondary,no,yes,no,unknown,may,unknown,no,1,1,1,0,1,0,2,8,3,0


In [19]:
#загружаем le
with open('/content/job_le.pkl', 'rb') as f:
  job_le = pickle.load(f)

In [20]:
categorical_data['jov_inverse_transform'] = job_le.inverse_transform(categorical_data['jobLE'])
categorical_data.head()

,job,marital,education,default,housing,loan,contact,month,poutcome,y,...,maritalLE,educationLE,defaultLE,housingLE,loanLE,contactLE,monthLE,poutcomeLE,yLE,jov_inverse_transform
0,unemployed,married,primary,no,no,no,cellular,oct,unknown,no,...,1,0,0,0,0,0,10,3,0,unemployed
1,services,married,secondary,no,yes,yes,cellular,may,failure,no,...,1,1,0,1,1,0,8,0,0,services
2,management,single,tertiary,no,yes,no,cellular,apr,failure,no,...,2,2,0,1,0,0,0,0,0,management
3,management,married,tertiary,no,yes,yes,unknown,jun,unknown,no,...,1,2,0,1,1,2,6,3,0,management
4,blue-collar,married,secondary,no,yes,no,unknown,may,unknown,no,...,1,1,0,1,0,2,8,3,0,blue-collar


In [21]:
#удалим ненужную колонку
categorical_data.drop('jov_inverse_transform', axis = 1, inplace = True)
categorical_data.head(1)

,job,marital,education,default,housing,loan,contact,month,poutcome,y,jobLE,maritalLE,educationLE,defaultLE,housingLE,loanLE,contactLE,monthLE,poutcomeLE,yLE
0,unemployed,married,primary,no,no,no,cellular,oct,unknown,no,10,1,0,0,0,0,0,10,3,0


In [31]:
#удаляем исходные категориальные признаки
categorical_data.drop(categorical_cols, axis = 1, inplace = True)
categorical_data.head()

,jobLE,maritalLE,educationLE,defaultLE,housingLE,loanLE,contactLE,monthLE,poutcomeLE,yLE
0,10,1,0,0,0,0,0,10,3,0
1,7,1,1,0,1,1,0,8,0,0
2,4,2,2,0,1,0,0,0,0,0
3,4,1,2,0,1,1,2,6,3,0
4,1,1,1,0,1,0,2,8,3,0


#Числовые признаки

In [22]:
# список категориальных признаков
num_cols = data.describe().columns.to_list()
num_cols

['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']

In [24]:
num_data = data[num_cols].drop('age', axis = 1)
num_data.head()

,balance,day,duration,campaign,pdays,previous
0,1787,19,79,1,-1,0
1,4789,11,220,1,339,4
2,1350,16,185,1,330,1
3,1476,3,199,4,-1,0
4,0,5,226,1,-1,0


In [26]:
#standard scaler
scaler = StandardScaler()

num_data_scaled = scaler.fit_transform(num_data)
num_data_scaled[:3]

array([[ 0.12107186,  0.37405206, -0.7118608 , -0.57682947, -0.4072183 ,
        -0.32041282],
       [ 1.1186443 , -0.59602646, -0.16919389, -0.57682947,  2.98904408,
         2.04173372],
       [-0.02414438,  0.01027262, -0.30389844, -0.57682947,  2.89914302,
         0.27012381]])

In [28]:
#сохраняем scaler
with open('num_scaler.pkl', 'wb') as f:
  pickle.dump(scaler, f)

In [30]:
#проверяем scaler
scaler.transform([[1350,	16,	185,	1,	330,	1]])

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


array([[-0.02414438,  0.01027262, -0.30389844, -0.57682947,  2.89914302,
         0.27012381]])

In [35]:
num_cols[1:]

['balance', 'day', 'duration', 'campaign', 'pdays', 'previous']

In [36]:
num_data_scaled_df = pd.DataFrame(data = num_data_scaled, columns = num_cols[1:])
num_data_scaled_df.head()

,balance,day,duration,campaign,pdays,previous
0,0.121072,0.374052,-0.711861,-0.576829,-0.407218,-0.320413
1,1.118644,-0.596026,-0.169194,-0.576829,2.989044,2.041734
2,-0.024144,0.010273,-0.303898,-0.576829,2.899143,0.270124
3,0.017726,-1.566105,-0.250017,0.387967,-0.407218,-0.320413
4,-0.472753,-1.323585,-0.146102,-0.576829,-0.407218,-0.320413


#Собираем итоговый датасет

In [37]:
final_dataset = pd.concat([data['age'], categorical_data, num_data_scaled_df], axis = 1)
final_dataset.head()

,age,jobLE,maritalLE,educationLE,defaultLE,housingLE,loanLE,contactLE,monthLE,poutcomeLE,yLE,balance,day,duration,campaign,pdays,previous
0,30,10,1,0,0,0,0,0,10,3,0,0.121072,0.374052,-0.711861,-0.576829,-0.407218,-0.320413
1,33,7,1,1,0,1,1,0,8,0,0,1.118644,-0.596026,-0.169194,-0.576829,2.989044,2.041734
2,35,4,2,2,0,1,0,0,0,0,0,-0.024144,0.010273,-0.303898,-0.576829,2.899143,0.270124
3,30,4,1,2,0,1,1,2,6,3,0,0.017726,-1.566105,-0.250017,0.387967,-0.407218,-0.320413
4,59,1,1,1,0,1,0,2,8,3,0,-0.472753,-1.323585,-0.146102,-0.576829,-0.407218,-0.320413


#X-y, train_test_split

In [38]:
y = np.array(final_dataset['age'])
X = np.array(final_dataset.drop('age', axis = 1))

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.2,
                                                    shuffle = True,
                                                    random_state = 42)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(3616, 16)
(905, 16)
(3616,)
(905,)


#ML

In [39]:
kNNR = KNeighborsRegressor(n_neighbors=5)

kNNR.fit(X_train, y_train)


KNeighborsRegressor()

In [41]:
kNNR.predict(X_test[:5])

array([39.4, 39. , 41.4, 41. , 34.2])

In [42]:
y_test[:5]

array([51, 50, 50, 37, 31])

In [44]:
#Random forest regressor
fr = RandomForestRegressor()
rfr.fit(X_train, y_train)


RandomForestRegressor()

In [45]:
rfr.predict(X_test[:5])

array([45.91, 50.19, 42.4 , 40.37, 34.39])

In [46]:
y_test[:5]

array([51, 50, 50, 37, 31])

In [49]:
#Сохраняем модель
with open('RFR.pkl', 'wb') as f:
  pickle.dump(rfr, f)

In [50]:
#загружаем моедль и проверяем
with open('RFR.pkl', 'rb') as f:
  rfr_loaded = pickle.load(f)

In [51]:
rfr_loaded.predict(X_test[:5])

array([45.91, 50.19, 42.4 , 40.37, 34.39])

In [52]:
!pip list

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
aiohttp                          3.8.5
aiosignal                        1.3.1
alabaster                        0.7.13
albumentations                   1.3.1
altair                           4.2.2
anyio                            3.7.1
appdirs                          1.4.4
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array-record                     0.4.1
arviz                            0.15.1
astropy                          5.3.3
astunparse                       1.6.3
async-timeout                    4.0.3
attrs                            23.1.0
audioread                        3.0.0
autograd                         1.6.2
Babel                            2.12.1
backcall                         0.2.0
beautifulsoup4                   4.11.2
bleach                           6.0.0
blinker                          1.4
bl